In [ ]:
!pip install streamlit pyngrok langchain langchain-groq chromadb sentence-transformers pymupdf python-dotenv --upgrade langchain langchain-community langchain-groq chromadb sentence-transformers pymupdf python-dotenv pyngrok streamlit pdfplumber

In [ ]:
!pip install pandas matplotlib

In [ ]:
%%writefile app.py
import streamlit as st
import pdfplumber
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI

# Inisialisasi client Groq/OpenAI
client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)

st.set_page_config(page_title="CV Analyzer AI", layout="wide")
st.title("📄 CV Analyzer AI")

# Tabs
tab1, tab2, tab3, tab4, tab5 = st.tabs(["📤 Upload CV", "🔍 Ringkasan", "🤖 Rekomendasi", "⚖️ Perbandingan", "📊 Skoring CV"])

# Tab Upload CV
with tab1:
    st.header("Upload Beberapa CV (PDF) 📂")
    uploaded_files = st.file_uploader("Upload beberapa file PDF di sini", type=["pdf"], accept_multiple_files=True)
    if uploaded_files:
        st.session_state["cv_texts"] = []
        for file in uploaded_files:
            with pdfplumber.open(file) as pdf:
                text = "".join([page.extract_text() or "" for page in pdf.pages])
                st.session_state["cv_texts"].append({"filename": file.name, "text": text})
        st.success(f"✅ {len(uploaded_files)} CV berhasil diproses!")

# Tab Ringkasan
with tab2:
    st.header("Ringkasan CV 📝")
    if "cv_texts" in st.session_state:
        for i, cv in enumerate(st.session_state["cv_texts"]):
            with st.expander(f"{cv['filename']}"):
                with st.spinner("Meringkas..."):
                    response = client.chat.completions.create(
                        model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        messages=[
                            {"role": "system", "content": "Ringkas CV berikut ini dalam bahasa Indonesia, fokus pada pengalaman, keahlian, dan pencapaian utama."},
                            {"role": "user", "content": cv["text"]}
                        ]
                    )
                    summary = response.choices[0].message.content
                    st.write(summary)
    else:
        st.warning("⬅️ Silakan upload CV terlebih dahulu.")

# Tab Rekomendasi
with tab3:
    st.header("Rekomendasi AI 🤖")
    if "cv_texts" in st.session_state:
        for i, cv in enumerate(st.session_state["cv_texts"]):
            if st.button(f"Dapatkan Rekomendasi untuk {cv['filename']}"):
                with st.spinner("AI sedang menganalisis..."):
                    response = client.chat.completions.create(
                        model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        messages=[
                            {"role": "system", "content": "Kamu adalah HR profesional. Berikan rekomendasi dalam bahasa Indonesia terhadap CV ini."},
                            {"role": "user", "content": cv["text"]}
                        ]
                    )
                    result = response.choices[0].message.content
                    st.success("✅ Rekomendasi:")
                    st.write(result)
    else:
        st.warning("⬅️ Silakan upload CV terlebih dahulu.")

# Tab Perbandingan
with tab4:
    st.header("Perbandingan CV berdasarkan Role ⚖️")
    if "cv_texts" in st.session_state:
        role = st.text_input("Masukkan posisi atau role pekerjaan (misal: Data Analyst)")
        if st.button("Bandingkan CV"):
            with st.spinner("AI sedang membandingkan..."):
                cv_list_text = "\n\n".join([f"CV {i+1} ({cv['filename']}):\n{cv['text']}" for i, cv in enumerate(st.session_state["cv_texts"])])
                prompt = (
                    f"Berikut ini adalah beberapa CV. Bandingkan dan tentukan CV mana yang paling cocok untuk role '{role}'. "
                    "Jelaskan alasanmu dalam bahasa Indonesia:\n\n" + cv_list_text
                )
                response = client.chat.completions.create(
                    model="meta-llama/llama-4-maverick-17b-128e-instruct",
                    messages=[
                        {"role": "system", "content": "Kamu adalah HR profesional yang diminta membandingkan beberapa CV."},
                        {"role": "user", "content": prompt}
                    ]
                )
                comparison = response.choices[0].message.content
                st.success("✅ Hasil Perbandingan:")
                st.write(comparison)
    else:
        st.warning("⬅️ Silakan upload CV terlebih dahulu.")

# Tab Skoring CV
with tab5:
    st.header("📊 Visualisasi Skoring CV")

    if "cv_texts" in st.session_state:
        st.subheader("Kriteria Penilaian")
        col1, col2 = st.columns(2)

        with col1:
            role_scoring = st.text_input("Posisi yang dinilai (untuk penyesuaian skoring)")

        with col2:
            min_experience = st.number_input("Pengalaman minimal yang diharapkan (tahun)", min_value=0, max_value=20, value=2)

        if st.button("💯 Hitung Skor CV"):
            with st.spinner("Sedang mengevaluasi CV..."):
                scoring_results = []

                scoring_prompt = f"""
                Anda adalah HR profesional yang akan menilai CV berdasarkan kriteria berikut:
                1. Relevansi Pengalaman (0-30): Sesuai dengan posisi {role_scoring}
                2. Kualitas Pendidikan (0-20): Tingkat pendidikan dan reputasi institusi
                3. Keahlian Teknis (0-25): Keterampilan khusus yang relevan
                4. Prestasi (0-15): Pencapaian yang berdampak
                5. Kesesuaian Gaji (0-10): Level pengalaman vs ekspektasi gaji

                Minimum pengalaman yang diharapkan: {min_experience} tahun

                Berikan penilaian dalam format JSON dengan keys:
                - nama_file
                - relevansi_pengalaman
                - kualitas_pendidikan
                - keahlian_teknis
                - prestasi
                - kesesuaian_gaji
                - total_skor
                - catatan

                Berikan penilaian objektif dan jangan terlalu tinggi.
                Total skor maksimal adalah 100.
                """

                for cv in st.session_state["cv_texts"]:
                    response = client.chat.completions.create(
                        model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        response_format={"type": "json_object"},
                        messages=[
                            {"role": "system", "content": scoring_prompt},
                            {"role": "user", "content": cv["text"]}
                        ]
                    )

                    try:
                        result = eval(response.choices[0].message.content)
                        result["nama_file"] = cv["filename"]
                        scoring_results.append(result)
                    except:
                        st.error(f"Gagal memproses {cv['filename']}")

                df_scores = pd.DataFrame(scoring_results).sort_values("total_skor", ascending=False)

                col1, col2 = st.columns(2)

                with col1:
                    st.subheader("🏆 Ranking CV")
                    st.dataframe(df_scores.set_index("nama_file")[["total_skor"]].style.background_gradient(cmap="Blues"), use_container_width=True)

                    csv = df_scores.to_csv(index=False).encode("utf-8")
                    st.download_button("📥 Download Hasil Skoring (CSV)", csv, "hasil_skoring_cv.csv", "text/csv")

                with col2:
                    st.subheader("📈 Distribusi Skor")
                    fig, ax = plt.subplots()
                    df_scores["total_skor"].plot(kind="hist", bins=10, ax=ax, color="skyblue")
                    ax.set_xlabel("Total Skor")
                    ax.set_ylabel("Jumlah CV")
                    ax.set_title("Distribusi Total Skor CV")
                    st.pyplot(fig)

                st.subheader("📊 Perbandingan Kategori Skor")
                top_cvs = df_scores.head(3)["nama_file"].tolist()
                selected_cvs = st.multiselect("Pilih CV untuk dibandingkan", options=df_scores["nama_file"].tolist(), default=top_cvs)

                if len(selected_cvs) >= 2:
                    categories = ["relevansi_pengalaman", "kualitas_pendidikan", "keahlian_teknis", "prestasi", "kesesuaian_gaji"]
                    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

                    for cv_name in selected_cvs:
                        cv_data = df_scores[df_scores["nama_file"] == cv_name].iloc[0]
                        values = [cv_data[cat] for cat in categories] + [cv_data[categories[0]]]
                        angles = [n / float(len(categories)) * 2 * 3.14159 for n in range(len(categories))] + [0]

                        ax.plot(angles, values, linewidth=1, linestyle="solid", label=cv_name)
                        ax.fill(angles, values, alpha=0.1)

                    ax.set_thetagrids([angle * 180/3.14159 for angle in angles[:-1]], categories)
                    ax.set_title("Perbandingan Kategori Skor", y=1.1)
                    ax.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
                    st.pyplot(fig)
                else:
                    st.warning("Pilih minimal 2 CV untuk dibandingkan.")

                st.subheader("📝 Catatan Evaluasi")
                for index, row in df_scores.iterrows():
                    with st.expander(f"Catatan untuk {row['nama_file']} (Skor: {row['total_skor']}/100)"):
                        st.write(row["catatan"])
    else:
        st.warning("⬅️ Silakan unggah CV terlebih dahulu.")


Overwriting app.py


In [ ]:
import subprocess
import os


os.chdir('/content')


process = subprocess.Popen(['streamlit', 'run', 'app.py'])


import time
time.sleep(3)


In [ ]:
!pip install pyngrok streamlit pdfplumber

from pyngrok import ngrok


ngrok.set_auth_token("2wMP3HUjMvfjjHt3HgIsn9riVCe_5qx8ygzAJo7ogTR7KxGvV")

public_url = ngrok.connect("http://localhost:8501")
print(public_url)

NgrokTunnel: "https://be79-35-229-209-90.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from pyngrok import ngrok

ngrok.kill()

In [ ]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.209.90:8501

  Stopping...


In [ ]:
import subprocess
import os
import time

os.chdir('/content')

# Jalankan Streamlit sekali saja
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Tunggu streamlit aktif
time.sleep(3)

!pip install pyngrok streamlit pdfplumber

from pyngrok import ngrok

# Autentikasi ngrok
ngrok.set_auth_token("2wMP3HUjMvfjjHt3HgIsn9riVCe_5qx8ygzAJo7ogTR7KxGvV")

# Pastikan connect ke port 8501
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://cefa-35-229-209-90.ngrok-free.app" -> "http://localhost:8501"
